<a href="https://colab.research.google.com/github/shivamsharma00/shivamsharma00/blob/main/classifying_images_of_clothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install and import dependencies**

we'll need Tensorflow datasets, an API that simplifies downloading and accessing datasets, and provide several sample datasets to work with. We're also using a few helper libraries.

In [1]:
!pip install -U tensorflow_datasets

     |████████████████████████████████| 4.0 MB 26.3 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [2]:
from __future__ import absolute_import, division, print_function

# Import Tensorflow and Tensorflow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds

# Helper libraries
import math 
import numpy as np
import matplotlib.pyplot as plt

# Improve progress bar display
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

print(tf.__version__)



2.7.0


# **Loading the datasets**


In [3]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_data, test_data = dataset['train'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling fashion_mnist-train.tfrecord...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling fashion_mnist-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
# loading the label for human readable results
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat'
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

# **Explore the data**
Let's explore the format of the dataset before training the model. The following shows there are 60,000 images in the training set, and 10000 images in the test set.

In [4]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print('Number of training examples: {}'.format(num_train_examples))
print('Number of test examples:     {}'.format(num_test_examples))


Number of training examples: 60000
Number of test examples:     10000


# **Preprocess the data**
The value of each pixel in the image data is an integer is in the range [0, 255]. For the model to work properly, these values need to be normalized to the range [0, 1]. So here we create a normalization function, and then apply it to each image in the test and train datasets.


In [6]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

# The map function applies the normalize function to each element in the train and test datasets.
train_data = train_data.map(normalize)
test_data = test_data.map(normalize)


# **Build the model**


---

## Setup the layers

In [7]:
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                             tf.keras.layers.Dense(128, activation=tf.nn.relu),
                             tf.keras.layers.Dense(10,  activation=tf.nn.softmax)
])

## Compile the model


*   Loss Function - An algorithm for measuring how far the model's output are from the desired output. The goal of training is this measure loss.
*   Optimizer - An algorithm for adjusting the inner parameters of the model in order to minimize loss.
*   Metrics - Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.



In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Train the model


In [10]:
BATCH_SIZE = 32
train_data = train_data.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_data = test_data.batch(BATCH_SIZE)

In [12]:
model.fit(train_data, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Epoch 1/5
1875/1875 [==============================] - 13s 4ms/step - loss: 0.4993 - accuracy: 0.8239
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3720 - accuracy: 0.8647
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3354 - accuracy: 0.8772
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3181 - accuracy: 0.8837
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2954 - accuracy: 0.8908


## Evaluate accuracy
We will compare how the model performs on the test dataset. Use all example we have in the test dataset to assess accuracy.

In [13]:
test_loss, test_accuracy = model.evaluate(test_data, steps=math.ceil(num_test_examples/32))
print('Accuracy on test datasets:', test_accuracy)

313/313 [==============================] - 1s 4ms/step - loss: 0.3762 - accuracy: 0.8628
Accuracy on test datasets: 0.8628000020980835
